In [1]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep
import twitter_credentials
import oauth2 as oauth

In [10]:
consumer=oauth.Consumer(key=twitter_credentials.CONSUMER_KEY,secret=twitter_credentials.CONSUMER_SECRET)
access_token=oauth.Token(key=twitter_credentials.ACCESS_TOKEN,secret=twitter_credentials.ACCESS_TOKEN_SECRET)
client=oauth.Client(consumer,access_token)
print(client)

In [4]:
user = 'realdonaldtrump'


In [5]:
user = user.lower()
output_file = '{}.json'.format(user)
output_file_short = '{}_short.json'.format(user)
compression = zipfile.ZIP_DEFLATED

In [11]:
with open('updated_ids.json') as f:
    ids = json.load(f)

print('total ids: {}'.format(len(ids)))
print(ids)

total ids: 20
['5c472b55ab09a13be42c2be7', '5c472b55ab09a13be42c2be8', '5c472b55ab09a13be42c2be9', '5c472b55ab09a13be42c2bea', '5c472b55ab09a13be42c2beb', '5c472b55ab09a13be42c2bec', '5c472b55ab09a13be42c2bed', '5c472b55ab09a13be42c2bee', '5c472b55ab09a13be42c2bef', '5c472b55ab09a13be42c2bf0', '5c472b55ab09a13be42c2bf1', '5c472b55ab09a13be42c2bf2', '5c472b55ab09a13be42c2bf3', '5c472b55ab09a13be42c2bf4', '5c472b55ab09a13be42c2bf5', '5c472b55ab09a13be42c2bf6', '5c472b55ab09a13be42c2bf7', '5c472b55ab09a13be42c2bf8', '5c472b55ab09a13be42c2bf9', '5c472b55ab09a13be42c2bfa']


In [15]:
all_data = []
start = 0
end = 100
limit = len(ids)
i = math.ceil(limit / 100)

In [16]:
for go in range(i):
    print('currently getting {} - {}'.format(start, end))
    sleep(6)  # needed to prevent hitting API rate limit
    id_batch = ids[start:end]
    start += 100
    end += 100
    # tweets = api.statuses_lookup(id_batch)
    url="https://api.twitter.com/1.1/statuses/show.json?id=5c472b55ab09a13be42c2be7&tweet_mode=extended"
    response,data=client.request(url)
    tweets=json.loads(data)
    for tweet in tweets:
        all_data.append(tweet)
        print(tweet)

currently getting 0 - 100
errors


In [ ]:
print('metadata collection complete')
print('creating master json file')
with open(output_file, 'w') as outfile:
    json.dump(all_data, outfile)

In [ ]:
print('creating ziped master json file')
zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
zf.write(output_file, compress_type=compression)
zf.close()

In [ ]:
def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

with open(output_file) as json_data:
    data = json.load(json_data)
    for entry in data:
        t = {
            "created_at": entry["created_at"],
            "text": entry["text"],
            "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
            "retweet_count": entry["retweet_count"],
            "favorite_count": entry["favorite_count"],
            "source": get_source(entry),
            "id_str": entry["id_str"],
            "is_retweet": is_retweet(entry)
        }
        results.append(t)

In [ ]:
print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    json.dump(results, outfile)

with open(output_file_short) as master_file:
    data = json.load(master_file)
    fields = ["favorite_count", "source", "text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]
    print('creating CSV version of minimized json master file')
    f = csv.writer(open('{}.csv'.format(user), 'w'))
    f.writerow(fields)
    for x in data:
        f.writerow([x["favorite_count"], x["source"], x["text"], x["in_reply_to_screen_name"], x["is_retweet"], x["created_at"], x["retweet_count"], x["id_str"]])
